<a href="https://colab.research.google.com/github/Rainshield/COMP551/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import files    
import warnings
warnings.filterwarnings('ignore')  
    

In [ ]:
class reg_dat:
    
      
       
    '''
    Regression data class. The class loads the data, calculates the statistical summaries, 
    prints, and saves them if needed. The esplit function splits the data into train-test
    parts based on a given ratio. If 'normalize' method is raised, z-score normalized 
    versions of train and test set are created and stored in the class. 
    
    pth: Path to the data location
    '''
    def __init__(self, pth):
        self.dat     = pd.read_excel(pth)
        self.n       = self.dat.shape[0]
        self.x_train = None
        self.y_train = None
        self.x_test  = None     
        self.y_test  = None
             
    def stat_sum(self, prnt, save): #Provide statistical summary of the data
        stat_sum = pd.DataFrame(self.dat.describe())
        cat_1 = (self.dat['X8'].value_counts()/self.n) * 100 #Categorical data
        cat_2 = (self.dat['X6'].value_counts()/self.n) * 100
        cat_3 = (self.dat['X4'].value_counts()/self.n) * 100
        cat_4 = (self.dat['X5'].value_counts()/self.n) * 100
        cat_5 = (self.dat['X7'].value_counts()/self.n) * 100
        all_cat = pd.concat([cat_1, cat_2, cat_3, cat_4, cat_5], axis=1)
        if prnt:
            print(stat_sum)
            print('Categorical data:')
            print(all_cat)
        if save: 
            stat_sum.to_excel('statistical_summary.xlsx')
            all_cat.to_excel('categorical_statistical_summary.xlsx')
           
    def split(self, tst_ratio, outcm=9, rnd_stat=100):        
        # Split into train test sets. Note that the outcomes are Y1 (8), Y2 (9)
        if not( outcm == 9 or outcm ==8):
            print('Warning! Your choosing an independent variable as outcome!')
            
        x_train, x_test, y_train, y_test = train_test_split(self.dat.iloc[:, 0:8],
                                                               self.dat.iloc[:, outcm],
                                                               test_size=tst_ratio,
                                                               random_state=rnd_stat)
        self.x_train = x_train.values
        self.x_test  = x_test.values
        self.y_train = y_train.values
        self.y_test  = y_test.values

           
    def normalize(self): # zscore normalization
           
        sclr = StandardScaler()
        self.nx_train = sclr.fit_transform(self.x_train)
        self.nx_test  = sclr.transform(self.x_test)
           
    def normalize_mn(self): #Min max normalization (scale all features to range(0,1))
        sclr = MinMaxScaler()
        self.nnx_train = sclr.fit_transform(self.x_train)
        self.nnx_test  = sclr.transform(self.x_test)
              

#Helper functions for:
## 1- evaluating linear regression models via MSE and R2 score
## 2- The gradient function used for gradient descent

In [ ]:
# Helper functions for:
# 1- evaluating linear regression models via MSE and R2 score
# 2- The gradient function used for gradient descent

def reg_eval(y, y_pred):
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    print(f'MSE: {mse}')
    print(f'R2 score: {r2}')
    return mse, r2



def gradient( x, y, w, alpha=0):
    y_pred =  x @ w 
    dlt_y = y_pred - y
    N, _ = x.shape

    
    grad = (.5*np.dot(dlt_y.T, x)/N) + alpha * w.T
    # delta y is inversed to adjust the dimensions for multiplication
    return grad.T # To convert into (D,1) Format



# The bellow cell contains:
### 1- The GradDescent class for performing gradient descent optimization
### 2- The Linear Regression class that fits the data using least square and 
### gradient descent

In [ ]:
# This cell contains:
# 1- The GradDescent class for performing gradient descent optimization
# 2- The Linear Regression class that fits the data using least square and 
# gradient descent

class GradDescent:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8,
                 btch_sz=32, tol=5, alpha=0.01, record_history=False):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.record_history = record_history
        self.epsilon = epsilon
        self.btch_sz = btch_sz # batch size
        self.alpha = alpha # For L2 regularization
        self.tol = tol # Tolerance
        if record_history:
            self.w_history = []                 #to store the weight history for visualization
            
    

    def run(self, gradient_fn, x, y, w):        
        
        grad = np.inf
        tol  = 0    #tolerance tracking index
        btch = 0    #batch tracking index
        N = x.shape[0]
        btch_num = np.floor(N/self.btch_sz) # Number of full batches (e.g., a dataset
        # with length of 10 and batch size of 3 will have 3 full batches)
        
        #--------- Initialize a list to store losses in each iteration
        tmp_ls = []
        #-----------
        for ind in range(int(self.max_iters)):
            if ind%100 == 0: # print iteration
                print(f'Iteration: {ind}')
                
            if np.linalg.norm(grad) < self.epsilon: # If grad goes bellow the epsilon
                if tol >= self.tol: # if the grad had small changes for tol iterations
                    break
                else:
                    tol+=1
            
            if btch == 0:    # Batch initiation. if batch tracker is zero, get 
            # a shuffled index set to choose from x and y instances when each batch 
            # is being processes
                tmp_ind = np.random.permutation(np.arange(N))
            
            if btch==btch_num: # for the last batch, most of the times its length
            # is not equal to the batch size (is smaller), and this can cause problem
            # if you use routine indexing
                slc_ind = tmp_ind[(btch)*self.btch_sz:]
                x_bt = x[slc_ind, :]
                y_bt = y[slc_ind,:]

                btch = 0 # Reset batch tracking index
            else:
                slc_ind = tmp_ind[(btch)*self.btch_sz:(btch+1)*self.btch_sz]
                x_bt = x[slc_ind, :]
                y_bt = y[slc_ind, :]

                btch += 1
            
            grad = gradient_fn(x_bt, y_bt, w, self.alpha)  #Calculate gradient 
           
            w = w - self.learning_rate * grad # Take the step
            # ------------- calculate and store loss -----------
            tmp_y = x @ w
            tmp_ls.append(mean_squared_error(y, tmp_y)) #Store loss in each iteration
            #----------------
            if self.record_history:
                self.w_history.append(w)
                
            if tol>0  and np.linalg.norm(grad) > self.epsilon: # reset tolerance if gradient goes over epsilon
                tol=0 
            
        return w, tmp_ls
       



#%%



class LinearReg:
    '''
    The linear regression class. The class  performes training and prediction with 
    least square and gradient discent. 
    
    '''
    
    
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        
    def fit_ls(self, x, y):
        if x.ndim == 1:
            x = x[:, None]                         #add a dimension for the features
        N = x.shape[0]
        if self.add_bias:
            x = np.column_stack([x,np.ones(N)])    #add bias by adding a constant feature of value 1
        #alternatively: self.w = np.linalg.inv(x.T @ x)@x.T@y
        self.w = np.linalg.lstsq(x, y)[0]          #return w for the least square difference
        return self  
    
    
    def predict_ls(self, x):     #least square predictinon
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])

        y_pred = x @ self.w                             #predict the y values
        return y_pred



    def fit_grd(self, x, y, optimizer, gradient):   #grad descent fitting
        if x.ndim == 1:
            x = x[:, None]                         #add a dimension for the features
        N, _ = x.shape
        if self.add_bias:
            x = np.column_stack([x,np.ones(N)])
        
        _, D = x.shape
        # w0 = np.zeros((D,1))
        w0 = np.random.randn(D,1)    #Initialize weighs with random normal values
        self.w_grd, self.tmp_ls = optimizer.run(gradient, x, y, w0 ) 
        
      
            
    def predict_grd(self, x):   # gradient descent prediction
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])

        y_pred = x @ self.w_grd                             #predict the y values
        return y_pred

# The bellow cell will:
Upload the data. 
Create the regression data class.
Calculate and print statistical summries for variables. X6 and X8 seem to be categorical though a few others (like X5) also have few unique values

In [ ]:
# Need to upload the data into colab first
uploaded = files.upload()

pth = '/ENB2012_data.xlsx' # Path to load the data
rg_dat = reg_dat(pth) # The regression data instance
rg_dat.stat_sum(prnt=True, save=False) #

#


Saving ENB2012_data.xlsx to ENB2012_data.xlsx
               X1          X2          X3          X4         X5          X6  \
count  768.000000  768.000000  768.000000  768.000000  768.00000  768.000000   
mean     0.764167  671.708333  318.500000  176.604167    5.25000    3.500000   
std      0.105777   88.086116   43.626481   45.165950    1.75114    1.118763   
min      0.620000  514.500000  245.000000  110.250000    3.50000    2.000000   
25%      0.682500  606.375000  294.000000  140.875000    3.50000    2.750000   
50%      0.750000  673.750000  318.500000  183.750000    5.25000    3.500000   
75%      0.830000  741.125000  343.000000  220.500000    7.00000    4.250000   
max      0.980000  808.500000  416.500000  220.500000    7.00000    5.000000   

               X7         X8          Y1          Y2  
count  768.000000  768.00000  768.000000  768.000000  
mean     0.234375    2.81250   22.307195   24.587760  
std      0.133221    1.55096   10.090204    9.513306  
min      0.00

In [ ]:
#Split into train test with a ratio of 0.2. Outcome can be 9 (Y2) or 8 (Y1)
rg_dat.split(tst_ratio=0.2, outcm=8, rnd_stat=100) #Split into train test with a ratio of 0.2
rg_dat.normalize() # zscore normalize the data


In [ ]:
linrg = LinearReg() #Create linear regression instance
linrg.fit_ls(rg_dat.nx_train, rg_dat.y_train) #fit least square (dat.nx_train is  z score normalized data)

y_prd = linrg.predict_ls(rg_dat.nx_test) # predict using least square coefs
y_prd_tr = linrg.predict_ls(rg_dat.nx_train)

reg_eval(rg_dat.y_test, y_prd) #test set evaluation

reg_eval(rg_dat.y_train, y_prd_tr) #training set evaluation

MSE: 8.441431556980932
R2 score: 0.9094940323320938
MSE: 8.55119792801829
R2 score: 0.917609840188006


<ipython-input-5-c904f89bf584>:104: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self.w = np.linalg.lstsq(x, y)[0]          #return w for the least square difference


(8.55119792801829, 0.917609840188006)

In [ ]:
optmz = GradDescent(learning_rate=.005, max_iters=1e5, epsilon=1e-4,
             btch_sz=128, alpha=0, tol=5) # Optimizer. Alpha tunes L2 regularization power


linrg.fit_grd(rg_dat.nx_train, rg_dat.y_train[:, None], optmz, gradient) #Fit


In [ ]:
y_pred = linrg.predict_grd(rg_dat.nx_test)# Predict
reg_eval(rg_dat.y_test, y_pred) #test set evaluation

y_prd_tr = linrg.predict_grd(rg_dat.nx_train)
reg_eval(rg_dat.y_train, y_prd_tr) #train set evaluation


MSE: 8.512514887106677
R2 score: 0.9087319026465468
MSE: 8.57618675039632
R2 score: 0.9173690747319178


(8.57618675039632, 0.9173690747319178)